In [ ]:
import arcpy
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import (
    confusion_matrix,
    classification_report,
    accuracy_score,
    cohen_kappa_score,
)
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from collections import Counter

# Set environment settings
print("Setting up environment...")
arcpy.env.workspace = r"D:\YAP\RandomForest"
arcpy.env.overwriteOutput = True

# Define paths
print("Defining paths...")
input_raster = r"D:\YAP\RandomTree\Sentinel2_2017_TWD.tif"
reference_raster = os.path.join(r"D:\YAP\RandomTree", "reference_raster.tif")
output_folder = r"D:\YAP\RandomTree"

# Output paths
accuracy_report_path = os.path.join(output_folder, "accuracy_report_2017.txt")
csv_output_path = os.path.join(output_folder, "accuracy_report_2017.csv")
heatmap_output_path = os.path.join(output_folder, "confusion_matrix_heatmap_2017.png")
training_samples_shapefile = os.path.join(output_folder, "training_samples_2017.shp")
testing_samples_shapefile = os.path.join(output_folder, "testing_samples_2017.shp")

# Read raster data
print("Reading raster data...")
raster = arcpy.RasterToNumPyArray(input_raster)
raster_shape = raster.shape  # (bands, rows, columns)
desc = arcpy.Describe(input_raster)
lower_left = desc.extent.lowerLeft

# Get cell size using arcpy.GetRasterProperties_management
cell_width = float(
    arcpy.GetRasterProperties_management(input_raster, "CELLSIZEX").getOutput(0)
)
cell_height = float(
    arcpy.GetRasterProperties_management(input_raster, "CELLSIZEY").getOutput(0)
)

# Read the reference raster
print("Reading reference raster...")
reference = arcpy.RasterToNumPyArray(reference_raster)

# Function to generate samples from the raster and reference raster
def generate_samples_from_raster(
    raster_array, reference_array, num_samples_per_class, output_points_path
):
    print("Generating samples from raster and reference raster...")
    # Reshape raster data to 2D array (pixels x bands)
    raster_2d = raster_array.reshape(raster_array.shape[0], -1).T
    # Flatten reference raster labels
    reference_flat = reference_array.ravel().astype(str)

    # Get unique class labels
    classes, counts = np.unique(reference_flat, return_counts=True)
    print("Classes in reference raster:", classes)

    # Initialize lists to hold samples and labels
    data_list = []
    labels_list = []
    x_coords = []
    y_coords = []

    for cls in classes:
        if cls == "nan" or cls == "":
            continue  # Skip invalid classes
        # Get indices of pixels belonging to the current class
        indices = np.where(reference_flat == cls)[0]
        if len(indices) == 0:
            continue
        # Randomly select samples from the indices
        np.random.shuffle(indices)
        selected_indices = indices[:num_samples_per_class]

        # Append the selected samples and labels
        data_list.append(raster_2d[selected_indices])
        labels_list.append(np.full(len(selected_indices), cls))

        # Calculate coordinates for the selected indices
        rows = selected_indices // raster_shape[2]
        cols = selected_indices % raster_shape[2]
        x = lower_left.X + cols * cell_width + cell_width / 2
        y = lower_left.Y + rows * cell_height + cell_height / 2
        x_coords.extend(x)
        y_coords.extend(y)

    # Combine data and labels
    if data_list:
        data = np.vstack(data_list)
        labels = np.hstack(labels_list)
        coords = np.column_stack((x_coords, y_coords))
    else:
        data = np.array([])
        labels = np.array([])
        coords = np.array([])

    # Export samples to shapefile
    if coords.size > 0:
        print(f"Exporting samples to shapefile: {output_points_path}")
        spatial_ref = desc.spatialReference

        # Create a NumPy array with fields for coordinates and labels
        samples_array = np.array(
            list(zip(coords[:, 0], coords[:, 1], labels)),
            dtype=[("X", "f8"), ("Y", "f8"), ("Label", "U50")],
        )

        # Define the in-memory table path
        in_memory_table = "in_memory\\samples_table"

        # Check if the in-memory table exists and delete it
        if arcpy.Exists(in_memory_table):
            arcpy.Delete_management(in_memory_table)

        # Convert NumPy array to a table
        arcpy.da.NumPyArrayToTable(samples_array, in_memory_table)

        # Create a point feature class from the table
        arcpy.management.XYTableToPoint(
            in_table=in_memory_table,
            out_feature_class=output_points_path,
            x_field="X",
            y_field="Y",
            coordinate_system=spatial_ref,
        )

        # Add and calculate the label field
        if 'Label' not in [f.name for f in arcpy.ListFields(output_points_path)]:
            arcpy.management.AddField(output_points_path, "Label", "TEXT")
        arcpy.management.CalculateField(
            output_points_path, "Label", "!Label!", "PYTHON3"
        )

        # Clean up in-memory table
        arcpy.Delete_management(in_memory_table)
    else:
        print("No samples to export.")

    return data, labels, coords  # Return coords as well

# Generate samples from raster
training_data, training_labels, coords = generate_samples_from_raster(
    raster,
    reference,
    num_samples_per_class=1000,  # Adjust as needed
    output_points_path=training_samples_shapefile,
)

# Encode labels
print("Encoding labels...")
le = LabelEncoder()
labels_str = training_labels.astype(str)
le.fit(labels_str)
labels_encoded = le.transform(labels_str)

# Split data into training and testing sets
print("Splitting data into training and testing sets...")
(
    training_data,
    testing_data,
    training_labels_encoded,
    testing_labels_encoded,
    training_coords,
    testing_coords,
    training_labels_str,
    testing_labels_str,
) = train_test_split(
    training_data,
    labels_encoded,
    coords,
    labels_str,
    test_size=0.2,
    stratify=labels_encoded,
    random_state=42,
)

# Export testing samples to shapefile
def export_samples_to_shapefile(coords, labels, output_points_path):
    print(f"Exporting samples to shapefile: {output_points_path}")
    spatial_ref = desc.spatialReference

    # Create a NumPy array with fields for coordinates and labels
    samples_array = np.array(
        list(zip(coords[:, 0], coords[:, 1], labels)),
        dtype=[("X", "f8"), ("Y", "f8"), ("Label", "U50")],
    )

    # Define the in-memory table path
    in_memory_table = "in_memory\\samples_table_test"

    # Check if the in-memory table exists and delete it
    if arcpy.Exists(in_memory_table):
        arcpy.Delete_management(in_memory_table)

    # Convert NumPy array to a table
    arcpy.da.NumPyArrayToTable(samples_array, in_memory_table)

    # Create a point feature class from the table
    arcpy.management.XYTableToPoint(
        in_table=in_memory_table,
        out_feature_class=output_points_path,
        x_field="X",
        y_field="Y",
        coordinate_system=spatial_ref,
    )

    # Add and calculate the label field
    if 'Label' not in [f.name for f in arcpy.ListFields(output_points_path)]:
        arcpy.management.AddField(output_points_path, "Label", "TEXT")
    arcpy.management.CalculateField(
        output_points_path, "Label", "!Label!", "PYTHON3"
    )

    # Clean up in-memory table
    arcpy.Delete_management(in_memory_table)

# Export testing samples
export_samples_to_shapefile(testing_coords, testing_labels_str, testing_samples_shapefile)

# Scale features
print("Scaling features...")
scaler = StandardScaler()
training_data_scaled = scaler.fit_transform(training_data)
testing_data_scaled = scaler.transform(testing_data)

# Hyperparameter tuning using RandomizedSearchCV
print("Starting hyperparameter tuning...")
param_dist = {
    'n_estimators': [100, 200, 300, 400, 500],
    'max_depth': [None, 10, 20, 30],
    'random_state': [0, 42, 100, 2021]
}

rf = RandomForestClassifier(
    class_weight='balanced',
    n_jobs=-1  # Utilize all available cores
)

random_search = RandomizedSearchCV(
    estimator=rf,
    param_distributions=param_dist,
    n_iter=10,  # Number of parameter settings that are sampled
    scoring='accuracy',
    cv=3,
    n_jobs=-1,
    verbose=2,
    random_state=42
)

random_search.fit(training_data_scaled, training_labels_encoded)
best_params = random_search.best_params_
print(f"Best parameters found: {best_params}")

# Train Random Forest Classifier with best parameters
print("Training Random Forest Classifier with best parameters...")
clf = RandomForestClassifier(
    n_estimators=best_params['n_estimators'],
    max_depth=best_params['max_depth'],
    class_weight='balanced',
    random_state=best_params['random_state'],
    n_jobs=-1
)
clf.fit(training_data_scaled, training_labels_encoded)

# Evaluate on testing data
print("Evaluating model on testing data...")
predictions = clf.predict(testing_data_scaled)
class_names = le.classes_
conf_matrix = confusion_matrix(testing_labels_encoded, predictions)
classification_rep = classification_report(
    testing_labels_encoded,
    predictions,
    target_names=class_names,
    zero_division=1
)
overall_accuracy = accuracy_score(testing_labels_encoded, predictions)
kappa_value = cohen_kappa_score(testing_labels_encoded, predictions)
print(f"Overall Accuracy: {overall_accuracy:.2f}")
print(f"Kappa Value: {kappa_value:.2f}")

# Save evaluation results
print("Saving evaluation results...")
with open(accuracy_report_path, 'w') as f:
    f.write("Evaluation on Testing Data:\n")
    f.write("Confusion Matrix:\n")
    f.write(str(conf_matrix))
    f.write("\n\nClassification Report:\n")
    f.write(classification_rep)
    f.write(f"\nOverall Accuracy: {overall_accuracy:.2f}\n")
    f.write(f"Kappa Value: {kappa_value:.2f}\n")

# Classify the entire raster using tile-based processing
print("Classifying the entire raster using tile-based processing...")

# Define tile size (rows, columns)
tile_size = (500, 500)  # Adjust as needed based on your system's memory capacity

# Get raster dimensions
n_bands, n_rows, n_cols = raster_shape

# Initialize an empty array to store the classified result
classified = np.zeros((n_rows, n_cols), dtype=np.float32)

# Loop over tiles
for row_start in range(0, n_rows, tile_size[0]):
    row_end = min(row_start + tile_size[0], n_rows)
    for col_start in range(0, n_cols, tile_size[1]):
        col_end = min(col_start + tile_size[1], n_cols)
        print(f"Processing tile: Rows {row_start}-{row_end}, Columns {col_start}-{col_end}")

        # Extract tile data
        tile_data = raster[:, row_start:row_end, col_start:col_end]
        tile_shape = tile_data.shape  # (bands, tile_rows, tile_cols)

        # Reshape tile data to 2D array (pixels x bands)
        tile_data_2d = tile_data.reshape(tile_shape[0], -1).T

        # Scale tile data
        tile_data_scaled = scaler.transform(tile_data_2d)

        # Predict classes for the tile
        tile_predictions = clf.predict(tile_data_scaled)

        # Reshape predictions back to tile shape and place in the classified array
        tile_predictions_2d = tile_predictions.reshape((row_end - row_start), (col_end - col_start))
        classified[row_start:row_end, col_start:col_end] = tile_predictions_2d.astype(np.float32)

# Save classified raster
print("Saving classified raster...")
output_raster_path = os.path.join(output_folder, "classified_image_2017.tif")
classified_raster = arcpy.NumPyArrayToRaster(classified, lower_left, cell_width, cell_height)
classified_raster.save(output_raster_path)

# Read the classified raster and reference raster
print("Reading classified raster and reference raster...")
classified_raster_array = arcpy.RasterToNumPyArray(output_raster_path)
reference_flat = reference.ravel().astype(str)
classified_flat = classified_raster_array.ravel()

# Convert classified labels to original class names
predicted_class_names = le.inverse_transform(classified_flat.astype(int))

# Map predicted labels to the same label encoding as reference labels
le_ref = LabelEncoder()
all_labels = np.concatenate((reference_flat, predicted_class_names))
le_ref.fit(all_labels)
reference_encoded = le_ref.transform(reference_flat)
predicted_encoded = le_ref.transform(predicted_class_names)

# Evaluate classification over the entire raster
print("Evaluating classification over the entire raster...")
overall_accuracy_full = accuracy_score(reference_encoded, predicted_encoded)
kappa_value_full = cohen_kappa_score(reference_encoded, predicted_encoded)
print(f"Overall Accuracy (full raster): {overall_accuracy_full:.2f}")
print(f"Kappa Value (full raster): {kappa_value_full:.2f}")

# Append results to the accuracy report
with open(accuracy_report_path, 'a') as f:
    f.write("\n\nEvaluation on Entire Raster:\n")
    f.write(f"\nOverall Accuracy: {overall_accuracy_full:.2f}\n")
    f.write(f"Kappa Value: {kappa_value_full:.2f}\n")

# Calculate area for each class
print("Calculating area for each class...")
cell_area_sqm = cell_width * cell_height
cell_area_sqkm = cell_area_sqm / 1e6  # Convert cell area to square kilometers

# Create DataFrames
reference_df = pd.DataFrame({'Class': reference_flat})
classified_df = pd.DataFrame({'Class': predicted_class_names})

# Calculate pixel counts
reference_counts = reference_df['Class'].value_counts()
classified_counts = classified_df['Class'].value_counts()

# Compute areas
reference_areas = (reference_counts * cell_area_sqkm).sort_index()
classified_areas = (classified_counts * cell_area_sqkm).sort_index()

# Combine into a DataFrame
area_comparison = pd.DataFrame({
    'Reference Area (sq km)': reference_areas,
    'Classified Area (sq km)': classified_areas
}).fillna(0)

# Calculate differences
area_comparison['Difference (sq km)'] = area_comparison['Classified Area (sq km)'] - area_comparison['Reference Area (sq km)']
area_comparison['% Difference'] = (area_comparison['Difference (sq km)'] / area_comparison['Reference Area (sq km)']) * 100

# Save the area comparison
area_comparison_path = os.path.join(output_folder, 'area_comparison_2017.csv')
area_comparison.to_csv(area_comparison_path, index_label='Class')
print(f"Area comparison saved to {area_comparison_path}")

# Plot and save the confusion matrix
print("Plotting confusion matrix...")
plt.figure(figsize=(10, 7))
sns.heatmap(
    conf_matrix,
    annot=True,
    fmt='d',
    cmap="Blues",
    xticklabels=class_names,
    yticklabels=class_names
)
plt.title('Confusion Matrix for 2017')
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.savefig(heatmap_output_path)
plt.close()
print(f"Confusion matrix saved to {heatmap_output_path}")

print("\nProcessing complete for the year 2017.")